### Imports

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

2. Sending HTTP request to access first page

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)

3. Checking code for http response

In [ ]:
response.status_code

4. Parse the HTML code of first page with opinions

In [ ]:
page_dom = BeautifulSoup(response.text, 'html.parser')

5. Extract required data from parsed code

In [ ]:
opinion = page_dom.select_one("div.js_product-review:not(.user-post--highlight)")
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

In [ ]:
for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recommendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        recommendation = None
    stars = opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    pros = [p.get_text() for p in (opinion.select("div.review-feature__item--positive"))]
    cons = [c.get_text() for c in (opinion.select("div.review-feature__item--negative"))]
    vote_yes = opinion.select_one("button.vote-yes")["data-total-vote"]
    vote_no = opinion.select_one("button.vote-no")["data-total-vote"]
    published_at = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
    try:
        purchased_at = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    except TypeError:
        purchased_at = None
    print(opinion_id, author, recommendation, stars, content, pros, cons, vote_yes, vote_no, published_at, purchased_at, sep='\n')

6. If there are more pages, move to the next page and repeat steps 2-6 for each one of them

In [ ]:
try:
    next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
except TypeError:
    next_page = None

In [ ]:
all_opinions = []
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        for opinion in opinions:
            single_opinion = {}
            single_opinion["option-id"] = opinion["data-entry-id"]
            single_opinion["author"] = opinion.select_one("span.user-post__author-name").get_text()
            try:
                single_opinion["recommendation"] = opinion.select_one("span.user-post__author-recomendation > em").get_text()
            except AttributeError:
                single_opinion["recommendation"] = None
            single_opinion["stars"] = opinion.select_one("span.user-post__score-count").get_text()
            single_opinion["content"] = opinion.select_one("div.user-post__text").get_text()
            single_opinion["pros"] = [p.get_text() for p in (opinion.select("div.review-feature__item--positive"))]
            single_opinion["cons"] = [c.get_text() for c in (opinion.select("div.review-feature__item--negative"))]
            single_opinion["vote_yes"] = opinion.select_one("button.vote-yes")["data-total-vote"]
            single_opinion["vote_no"] = opinion.select_one("button.vote-no")["data-total-vote"]
            single_opinion["published_at"] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
            try:
                single_opinion["purchased_at"] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
            except TypeError:
                single_opinion["purchased_at"] = None
            print(single_opinion, sep='\n')
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except AttributeError:
            next_page = None
    else:
        print('Something went wrong')

7. Save extracted data